In [0]:
%run ../../../utils

## Parametros y rutas

In [0]:
# Ruta base en tu Data Lake
silver_table_name = "silver.sales_countryregioncurrency"

## Lectura de Bronze

In [0]:
df_bronze = (
    spark.read.table("bronze.sales_countryregioncurrency ")
)

## Casting y estandarización de tipos

In [0]:
df_cast = (
    df_bronze
      .withColumn("CountryRegionCode", col("CountryRegionCode").cast("string"))
      .withColumn("CurrencyCode", col("CurrencyCode").cast("string"))
      .withColumn("ModifiedDate", col("ModifiedDate").cast("timestamp"))
)

## Limpieza básica

In [0]:
# Filtrar claves vacías
df_clean = df_cast.filter(
    col("CountryRegionCode").isNotNull() &
    col("CurrencyCode").isNotNull()
)

# Eliminar duplicados naturales
df_nodup = df_clean.dropDuplicates(["CountryRegionCode", "CurrencyCode"])

df_result= (
    df_nodup
      .withColumn("FechaAuditoriaCreacion", current_timestamp())
      .withColumn("FechaAuditoriaModificacion", current_timestamp())
)

## Merge incremental

In [0]:
# Ejecucion de merge, se pasa como tercer parametro la columna identity
merge(silver_table_name, df_result)